In [1]:
# import dependencies
import pandas as pd
import os

import sqlalchemy
from sqlalchemy import create_engine, MetaData
import pymysql
pymysql.install_as_MySQLdb()

# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [2]:
# read in cleaned data
measurement_path = os.path.join("Resources", "clean_hawaii_measurements.csv")
station_path = os.path.join("Resources", "clean_hawaii_stations.csv")

measurement_data = pd.read_csv(measurement_path)
station_data = pd.read_csv(station_path)

In [3]:
# create "hawaii" database
connection_string = "sqlite:///hawaii.sqlite"
engine = create_engine(connection_string)
conn = engine.connect()

In [9]:
# preview cleaned data
measurement_data.head()

,station,date,prcp,tobs
18098,USC00516128,2017-08-17,0.13,72
18099,USC00516128,2017-08-19,0.09,71
18100,USC00516128,2017-08-21,0.56,76
18101,USC00516128,2017-08-22,0.50,76
18102,USC00516128,2017-08-23,0.45,76


In [5]:
station_data.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [6]:
# create ORM classes for each table

class Measurement(Base):
    __tablename__ = 'measurement'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    date = Column(Text)
    prcp = Column(Float)
    tobs = Column(Integer)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"
    
class Station(Base):
    __tablename__ = 'station'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    name = Column(Text)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

Base.metadata.create_all(engine)

In [7]:
# write data to database tables

measurement_data_to_write = measurement_data.to_dict(orient='records')
station_data_to_write = station_data.to_dict(orient='records')

metadata = MetaData(bind=engine)
metadata.reflect()

measurement_table = sqlalchemy.Table('measurement', metadata, autoload=True)
station_table = sqlalchemy.Table('station', metadata, autoload=True)

conn.execute(measurement_table.insert(), measurement_data_to_write)
conn.execute(station_table.insert(), station_data_to_write)

In [8]:
# confirm data properly written to table

station_head = conn.execute("select * from station limit 5").fetchall()
measurement_head = conn.execute("select * from measurement limit 5").fetchall()

print("First 5 rows of station table:")
print(station_head)
print("First 5 rows of measurement table:")
print(measurement_head)

First 5 rows of station table:
[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0), (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6), (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0), (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9), (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]
First 5 rows of measurement table:
[(1, 'USC00519397', '2010-01-01', 0.08, 65), (2, 'USC00519397', '2010-01-02', 0.0, 63), (3, 'USC00519397', '2010-01-03', 0.0, 74), (4, 'USC00519397', '2010-01-04', 0.0, 76), (5, 'USC00519397', '2010-01-07', 0.06, 70)]
